In [ ]:
! pip install youtube-dl
! pip install git+https://github.com/Cupcakus/pafy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 12.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Cupcakus/pafy to /tmp/pip-req-build-0tb98w7f
  Running command git clone -q https://github.com/Cupcakus/pafy /tmp/pip-req-build-0tb98w7f
  Created wheel for pafy: filename=pafy-0.5.5-py2.py3-none-any.whl size=35757 sha256=396d76f6fcebc1b58b4f2d2bcbbd68b524337335419794e5b96752031105fd81
  Stored in directory: /tmp/pip-ephem-wheel-cache-ch5zaut2/wheels/93/1a/52/3c75e2f35b4aac122bcb0a64c08339224a76eafef3f150b9db
Successfully built pafy


In [ ]:
#yolo 학습을 위한 코드 불러오기
!git clone https://github.com/ultralytics/yolov5.git
#필요 라이브러리 설치
%cd /content/yolov5/
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 11983, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 11983 (delta 17), reused 25 (delta 17), pack-reused 11953
Receiving objects: 100% (11983/11983), 12.49 MiB | 24.13 MiB/s, done.
Resolving deltas: 100% (8241/8241), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 15.8 MB/s 


In [ ]:
import torch
import numpy as np
import cv2
import pafy
import time
import datetime
import yaml

In [ ]:
# Train YOLOv5s on VIsDrone for 3 epochs
! python train.py --img 640 --batch 16 --epochs 3 --data GlobalWheat2020.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=GlobalWheat2020.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-47-geab35f6 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_

In [ ]:
model1 = torch.hub.load("/content/yolov5", 'custom', path="yolov5s.pt", source='local')
model2 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

YOLOv5 🚀 v6.2-47-geab35f6 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)



  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v6.2-47-geab35f6 Python-3.7.13 torch-1.12.1+cu113 C

In [ ]:
class ObjectDetection:
    # YouTube 동영상에 YOLOv5 구현
    def __init__(self, model, url, log_file, out_file):
        # 객체 생성 시 호출
        # url: 예측 대상 YouTube URL
        # out_file: 유효한 출력 파일 이름 *.avi
        self._URL = url
        self.model = model
        self.classes = self.model.names
        self.out_file = out_file
        self.log_file = log_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    def get_video_from_url(self):
        # url에서 새 비디오 스트리밍 객체 생성
        play = pafy.new(self._URL).streams[-1]
        assert play is not None
        return cv2.VideoCapture(play.url)
    def score_frame(self, frame):
        # frame: 단일 프레임; numpy/list/tuple 형식
        # return: 프레임에서 모델이 감지한 객체의 레이블과 좌표
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].cpu().numpy(), results.xyxyn[0][:, :-1].cpu().numpy()
        return labels, cord
    def class_to_label(self, x):
        # x 숫자 레이블 -> 문자열 레이블로 반환
        return self.classes[int(x)]
    def plot_boxes(self, results, frame, File):
        # 경계상자와 레이블을 프레임에 플로팅
        # results: 프레임에서 모델이 감지한 객체의 레이블과 좌표
        # frame: 점수화된 프레임
        # return: 경계 상자와 레이블이 플로팅된 프레임
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]) + ': ' + str(x1) + ', ' + str(x2) + ', ' + str(y1) + ', ' + str(y2), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
                File.write(self.class_to_label(labels[i])+str(abs((x1-x2)*(y1-y2))))
        File.write("\n")
        return frame
    def frame_size(self, player):
        length = int(player.get(cv2.CAP_PROP_FRAME_COUNT))
        return length
    def __call__(self):
        # 인스턴스 생성 시 호출; 프레임 단위로 비디오 로드
        num = 0
        start = time.time()
        File = open(self.log_file,"w")
        player = self.get_video_from_url()
        assert player.isOpened()
        #영상 부분
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))
        #프레임수 조절
        # 프레임수 조절
        #length = self.frame_size(player)
        while  True:
            num+=1
            #player.set(cv2.CAP_PROP_POS_MSEC, 1000*num)
            num += 1
            ret, frame = player.read()
            if ret != True:
                break
            if num%2==1:
                continue
            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame, File)
            out.write(frame)
        File.close()
        print("time :", time.time() - start)

In [ ]:
Video = ObjectDetection(model1, "https://www.youtube.com/watch?v=qqVwHL4TFMk", "log.txt" ,"test.avi")
Video()

KeyboardInterrupt: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

x_train = [[10,5],[9,5],[8,5],[8,3],[8,2],[7,2],[7,1],[6,2],[3,2],[3,1]]
y_train = [0,0,0,1,1,1,1,1,2,2]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

model = nn.Linear(2,3)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

epoch = 1000
for i in range(epoch+1):
    prediction = model(x_train)
    cost = loss(prediction,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('예측값',prediction)
        print('x_train',x_train.shape)
        print('y_train',y_train.shape)

예측값 tensor([[ 1.9375,  0.6484, -4.4383],
        [ 1.5731,  0.7855, -3.7726],
        [ 1.2088,  0.9226, -3.1070],
        [ 1.8330,  0.2588, -3.9551],
        [ 2.1451, -0.0731, -4.3791],
        [ 1.7807,  0.0640, -3.7134],
        [ 2.0929, -0.2679, -4.1375],
        [ 1.4164,  0.2010, -3.0478],
        [ 0.3234,  0.6123, -1.0507],
        [ 0.6355,  0.2804, -1.4747]], grad_fn=<AddmmBackward0>)
x_train torch.Size([10, 2])
y_train torch.Size([10])
예측값 tensor([[-8.5357e-01,  2.7610e+00, -3.7599e+00],
        [-9.6756e-01,  2.7006e+00, -3.1471e+00],
        [-1.0815e+00,  2.6402e+00, -2.5342e+00],
        [-3.5922e-01,  1.9311e+00, -3.4351e+00],
        [ 1.9431e-03,  1.5765e+00, -3.8856e+00],
        [-1.1205e-01,  1.5161e+00, -3.2728e+00],
        [ 2.4912e-01,  1.1615e+00, -3.7232e+00],
        [-2.2604e-01,  1.4556e+00, -2.6599e+00],
        [-5.6801e-01,  1.2743e+00, -8.2130e-01],
        [-2.0685e-01,  9.1974e-01, -1.2718e+00]], grad_fn=<AddmmBackward0>)
x_train torch.Size([10, 2

KeyboardInterrupt: ignored

In [ ]:
target = torch.randint(5, (3,), dtype=torch.int64)
print(target.shape)
target2 = torch.empty(3, dtype=torch.long).random_(5)
print(target2.shape)

torch.Size([3])
torch.Size([3])
